In [3]:
%load_ext autoreload
%autoreload 2

%aimport xclimate

In [4]:
import xclimate as xclim

In [ ]:
GRID = xclim.load_fhist_ppe_grid()
NONGLC_PCT_THRESHOLD = 80
NONGLC_MASK = GRID.PCT_GLC < NONGLC_PCT_THRESHOLD

TIME_SLICE = slice("1995-01", "1996-12")

# Load variables
FHIST = {}
# FHIST["PRECT_day_1"] = xclim.load_coupled_fhist_ppe("PRECT", "atm", "day_1", "h2", keep_var_only=True)["PRECT"].sel(time=TIME_SLICE)
FHIST["PRECC_month_1"] = xclim.load_coupled_fhist_ppe("PRECC", "atm", "month_1", "h0", keep_var_only=True)["PRECC"].sel(time=TIME_SLICE)
FHIST["PRECL_month_1"] = xclim.load_coupled_fhist_ppe("PRECL", "atm", "month_1", "h0", keep_var_only=True)["PRECL"].sel(time=TIME_SLICE)
FHIST["TLAI_month_1"] = xclim.load_coupled_fhist_ppe("TLAI", "lnd", "month_1", "h0", keep_var_only=True)["TLAI"].sel(time=TIME_SLICE).reindex_like(GRID, method="nearest", tolerance=1e-3)
FHIST["ET_month_1"] = xclim.load_coupled_fhist_ppe("EFLX_LH_TOT", "lnd", "month_1", "h0", keep_var_only=True)["EFLX_LH_TOT"].sel(time=TIME_SLICE).reindex_like(GRID, method="nearest", tolerance=1e-3)

In [4]:
# Annual mean TLAI
FHIST["TLAI_year_1"] = FHIST["TLAI_month_1"].sel(time=TIME_SLICE).groupby("time.year").map(lambda x: x.weighted(x.time.dt.days_in_month).mean("time"))

# Annual mean ET
FHIST["ET_year_1"] = FHIST["ET_month_1"].sel(time=TIME_SLICE).groupby("time.year").map(lambda x: x.weighted(x.time.dt.days_in_month).mean("time"))

# Monthly precipitation rate (PRECC + PRECL)
FHIST["PRECT_month_1"] = FHIST["PRECC_month_1"] + FHIST["PRECL_month_1"]

# Annual difference between highest and lowest precipitation months
FHIST["PRDIFF_year_1"] = FHIST["PRECT_month_1"].groupby("time.year").map(lambda x: x.max(dim="time") - x.min(dim="time"))

# Wet-day frequency as defined in Feldman et al. (2024) - the annual number of days with above 1 mm/day of precipitation
# DAILY_PRECT_THRESH =  1 / (1000 * 24 * 60 * 60)  # [m/s] = 1 [mm/day]
# FHIST["WDFRQ_year_1"] = (FHIST["PRECT_day_1"].sel(time=TIME_SLICE).where(GRID>0) > DAILY_PRECT_THRESH).groupby("time.year").sum()

In [9]:
im = 1
nb = 15

result = xclim.get_quantile_binned_mean(
    Z=(FHIST["ET_year_1"].sel(member=im) - FHIST["ET_year_1"].sel(member=0)).where(NONGLC_MASK),
    xb=FHIST["PRDIFF_year_1"].sel(member=im).where(NONGLC_MASK),
    yb=FHIST["TLAI_year_1"].sel(member=im).where(NONGLC_MASK),
    xnb=nb,
    ynb=nb,
    quantile_dims=["lat", "lon", "year"],
    agg_dims=["gridcell"],
)

# result.binned_mean.plot()
# result.joint_hist.plot()